In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
import urllib.request
import gradio as gr

os.environ["OPENAI_API_KEY"] = "yourServiceKey"

c:\openai-prg\ch07\ch07_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = PyPDFLoader("2020_경제금융용어 700선_게시.pdf")
texts = loader.load_and_split()
print('문서의 수:', len(texts))

문서의 수: 366


In [3]:
texts[15]

Document(metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'page': 19, 'page_label': '20'}, page_content='3\nㄱ \n신용을 모두 포괄하는 것으로 크게 가계대출과 판매신용으로 구분된다. 현재 가계신용통\n계는 2002년 말 잔액부터 분기별로 제공되고 있다. 가계신용통계는 우리나라 가계부문의 \n부채 규모 및 변동 등을 파악하는 데 널리 활용되고 있다 .\n가계처분가능소득 \n가계처분가능소득(PDI; Personal Disposable Income)은 가계가 맘대로 소비와 저축\n으로 처분할 수 있는 소득을 의미한다. 흔히 국민들의 생활수준을 파악해 볼 수 있는 \n지표로 1인당 GNI가 널리 쓰이고 있으나 국민총소득에는 가계 뿐 아니라 기업 금융기관 \n정부가 벌어 들인 소득이 모두 포함되어 있다. 따라서 기업과 금융기관 등이 가계부문 \n보다 더 많은 소득을 벌어 1인당 국민총소득(GNI)이 높아진 경우에는 가계가 느끼는 \n체감경기는 전체 경기와 괴리가 있게 된다. 1인당 가계총처분가능소득(PGDI; Personal \nGross Disposable Income)은 가계부문의 총처분가능소득을 연앙인구로 나누

In [4]:
print(texts[13].page_content)

1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [5]:
texts = texts[13:]
print('줄어든 청크의 개수:', len(texts))

줄어든 청크의 개수: 353


In [6]:
print('첫번째 청크 출력 :', texts[0])

첫번째 청크 출력 : page_content='1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙' metadata={'producer': 'Smart PDF Imposition 1.0', 'creator': 'Smart PDF Imposition 1.0', 'creationdate': 'January 31, 2018 16:21', 'author': 'INSUNG DATA INC.', 'creationdate--text': 'January 31, 2018 16:21', 'moddate': 'January 31, 2018 16:21', 'moddate--text': 'January 31, 2018 16:21', 'subject': 'CoreImposition PDF 1.0 Output', 'ti

In [7]:
texts = texts[:-1]
print('마지막 데이터 제거 후 청크의 개수:', len(texts))

마지막 데이터 제거 후 청크의 개수: 352


In [8]:
print('텍스트 수 :', len(texts[-1].page_content))

텍스트 수 : 1107


In [9]:
embedding = OpenAIEmbeddings(chunk_size=199)

vectordb = Chroma.from_documents(documents=texts, embedding=embedding)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31652\3998786864.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(chunk_size=199)


In [10]:
print(vectordb._collection.count())

352


In [11]:
for key in vectordb._collection.get():
  print(key)

ids
embeddings
documents
uris
included
data
metadatas


In [12]:
documents = vectordb._collection.get()['documents']
print('청크의 개수 :', len(documents))
print('-' * 50)
print('0번 청크 출력 : ', documents[0])

청크의 개수 : 352
--------------------------------------------------
0번 청크 출력 :  1
ㄱ 
ㄱ
가계부실위험지수(HDRI)
가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 
부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면에서 평가하는 원리금상
환비율(DSR; Debt Service Ratio)과 자산 측면에서 평가하는 부채/자산비율(DTA; Debt 
To Asset Ratio)을 결합하여 산출한 지수이다. 가계부실위험지수는 가구의 DSR과 DTA가 
각각 40%, 100%일 때 100의 값을 갖도록 설정되어 있으며, 동 지수가 100을 초과하는 
가구를 ‘위험가구’로 분류한다. 위험가구는 소득 및 자산 측면에서 모두 취약한 ‘고위험가구’, 
자산 측면에서 취약한 ‘고DTA가구’, 소득 측면에서 취약한 ‘고DSR가구’로 구분할 수 있다. 
다만 위험 및 고위험 가구는 가구의 채무상환능력 취약성 정도를 평가하기 위한 것이며 
이들 가구가 당장 채무상환 불이행, 즉 임계상황에 직면한 것을 의미하지 않는다 .
 연관검색어 : 총부채원리금상환비율(DSR)
가계수지
가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 
표시한 것을 가계수지(household's total income and expenditure)라 한다. 가계수지가 
가계수지 ∙


In [13]:
embeddings = vectordb._collection.get(include=['embeddings'])['embeddings']
print('임베딩 벡터의 개수 :', len(embeddings))

임베딩 벡터의 개수 : 352


In [14]:
print('0번 청크의 임베딩 값 출력 : ', embeddings[0])
print('0번 청크의 임베딩 값 길이 : ', len(embeddings[0]))

0번 청크의 임베딩 값 출력 :  [-0.0077088  -0.0075166   0.014182   ... -0.01861645 -0.00669286
 -0.00903364]
0번 청크의 임베딩 값 길이 :  1536


In [15]:
metadatas = vectordb._collection.get()['metadatas']
print('metadatas의 개수 : ', len(metadatas))
print('0번 청크의 출처 :', metadatas[0])

metadatas의 개수 :  352
0번 청크의 출처 : {'source': '2020_경제금융용어 700선_게시.pdf', 'total_pages': 371, 'producer': 'Smart PDF Imposition 1.0', 'author': 'INSUNG DATA INC.', 'creator': 'Smart PDF Imposition 1.0', 'moddate--text': 'January 31, 2018 16:21', 'creationdate--text': 'January 31, 2018 16:21', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'page': 17, 'moddate': 'January 31, 2018 16:21', 'creationdate': 'January 31, 2018 16:21', 'page_label': '18', 'subject': 'CoreImposition PDF 1.0 Output'}


In [16]:
retriever = vectordb.as_retriever(kwargs=2)

docs = retriever.get_relevant_documents("비트코인이 궁금해")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('두번째 유사 문서 :', docs[1])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31652\1893799076.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("비트코인이 궁금해")


유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다

In [17]:
template = """
당신은 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이입니다.
안상준 개발자가 만들었습니다. 주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

In [18]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [19]:
qa_chain = RetrievalQA.from_chain_type(
  llm = llm,
  chain_type_kwargs={"prompt":prompt},
  retriever=retriever,
  return_source_documents=True
)

In [20]:
input_text = "디커플링이란 무엇인가?"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31652\440151274.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatbot_response = qa_chain(input_text)


{'query': '디커플링이란 무엇인가?', 'result': '디커플링은 탈동조화라고도 하는데, 어떤 나라나 지역의 경제가 인접한 다른 국가나 전반적인 세계 경제의 흐름과는 다른 모습을 보이는 현상을 말해. 예를 들어, 금융위기 이후 신흥국가나 유로지역 국가들이 특히 미국 경제와 다른 모습을 보이는 경우가 있어. 또, 주가나 금리, 환율 등 일부 경제 변수의 흐름이 국가 간 또는 특정 국가 내에서 서로 다른 흐름을 보이는 현상도 디커플링이라고 할 수 있어. 반대로, 경제가 다른 국가나 세계 경제 흐름과 유사한 흐름을 보이는 것을 커플링이라고 해.', 'source_documents': [Document(metadata={'total_pages': 371, 'subject': 'CoreImposition PDF 1.0 Output', 'moddate--text': 'January 31, 2018 16:21', 'creator': 'Smart PDF Imposition 1.0', 'source': '2020_경제금융용어 700선_게시.pdf', 'author': 'INSUNG DATA INC.', 'creationdate': 'January 31, 2018 16:21', 'producer': 'Smart PDF Imposition 1.0', 'title': '°æÁ¦±ÝÀ¶¿ë¾î 700¼±-ÃÖÁ¾.PDF', 'creationdate--text': 'January 31, 2018 16:21', 'page': 114, 'page_label': '115', 'moddate': 'January 31, 2018 16:21'}, page_content='98\n경제금융용어 700선\n가계와 기업이 소비와 투자를 미룸으로써 수요의 위축을 초래하여 디플레이션 압력을 \n보다 크게 할 수도 있다. 이 같은 상황에서는 경제주체들의 물가상승률 예상치가 더 \n낮아지지 않도록 사전에 통화 ･재정정책 면에서 적극적으로 대응하는 것이 중요하다 .\n 연관검색어 : 디플레이션,  통

In [21]:
def get_chatbot_response(input_text):
  chatbot_response = qa_chain(input_text)
  return chatbot_response['result'].strip()

In [22]:
input_text = '너는 뭘 하는 챗봇이니?'
result = get_chatbot_response(input_text)
print(result)

나는 한국은행에서 만든 금융 용어를 설명해주는 금융쟁이야. 안상준 개발자가 만들었고, 주어진 검색 결과를 바탕으로 금융 용어에 대해 친근하게 설명해줄 수 있어. 검색 결과에 없는 내용이라면 답변할 수 없어. 궁금한 게 있으면 물어봐!


In [23]:
input_text = '비트코인에 대해서 궁금하당~'
result = get_chatbot_response(input_text)
print(result)

비트코인은 가상통화이자 디지털 지급 시스템으로, 중앙 저장소나 단일 관리자가 없는 최초의 탈중앙화된 디지털 통화야. 사토시 나카모토라는 사람(또는 집단)이 2009년에 만들었고, 오픈 소스로 배포되었어. 비트코인 시스템은 P2P 네트워크를 통해 중개자 없이 거래가 이루어지고, 이 거래들은 블록체인에 기록돼. 채굴이라는 과정을 통해 비트코인이 발행되는데, 이 과정은 대규모 전기를 사용하는 컴퓨터 처리 능력을 필요로 해. 비트코인은 법정화폐가 아니고, 가격 변동이 심해서 투자 대상으로도 관심을 받고 있지만, 거품 논란도 있어. 일부 국가는 비트코인 거래를 규제하기도 해. 비트코인의 총량은 2100만 개로 한정되어 있고, 2140년경 모두 채굴될 것으로 예상돼.


In [24]:
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(label="경제금융용어 챗봇")
  msg = gr.Textbox(label="질문해주세요!")
  clear = gr.Button("대화 초기화")

  def respond(message, chat_history):
    result = qa_chain(message)
    bot_message = result['result']

    chat_history.append((message, bot_message))
    return "", chat_history
  
  msg.submit(respond, [msg, chatbot], [msg, chatbot])

  clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31652\3180202689.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="경제금융용어 챗봇")


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
